In [5]:
import os
from time import time
from pathlib import Path
import pandas as pd 
import numpy as np
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from datasets import Dataset
import accelerate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [6]:
# Obtenemos la carpeta actual
current_dir = Path.cwd()
DATASETS_LOCATION = os.path.join(current_dir.parent.parent.parent, 'datasets')
MODELS_LOCATION = os.path.join(current_dir.parent.parent, 'models')

Cargamos el dataset:

In [7]:
df_disease_sp = pd.read_excel(os.path.join(DATASETS_LOCATION, 'disease_nlp_esp.xlsx'))

# Mostrar el DataFrame
df_disease_sp.head(3)

,Síntomas,Enfermedad
0,Me siento cansado todo el tiempo y he notado q...,Anemia
1,Últimamente me falta el aire incluso después d...,Anemia
2,"Durante las últimas semanas, me siento extrema...",Anemia


In [9]:
print("Dataset Disease Spanish antes de undersampling:")
print(df_disease_sp['Enfermedad'].value_counts())


Dataset Language distribution antes de undersampling:
Enfermedad
Anemia       100
Talasemia    100
Trombosis    100
Diabetes     100
Name: count, dtype: int64


'# Determinamos la cantidad mínima de ejemplos por clase\nmin_count = min(df_spanish_english[\'Language\'].value_counts())\n\n# Verificamos el equilibrio\nprint("\n Dataset Language distribution despues de undersampling:")\nprint(df_balanced[\'Language\'].value_counts())'

Observamos que ya las clases están bien distribuidas. Dividimos el dataset en conjunto de entrenamiento y prueba:

In [10]:
# Dividiamo il dataset bilanciato in train (80%) e test (20%)
train_df, test_df = train_test_split(df_disease_sp, test_size=0.2, random_state=42, stratify=df_disease_sp['Enfermedad'])

# Dividiamo ulteriormente il train in train (80%) e validation (20%)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['Enfermedad'])

# Verifichiamo che i set siano equilibrati
print("\nConjunto de entrenamiento:")
print(train_df['Enfermedad'].value_counts())

print("\nConjunto de validación:")
print(val_df['Enfermedad'].value_counts())

print("\nConjunto de prueba:")
print(test_df['Enfermedad'].value_counts())



Conjunto de entrenamiento:
Enfermedad
Trombosis    64
Anemia       64
Talasemia    64
Diabetes     64
Name: count, dtype: int64

Conjunto de validación:
Enfermedad
Trombosis    16
Talasemia    16
Anemia       16
Diabetes     16
Name: count, dtype: int64

Conjunto de prueba:
Enfermedad
Talasemia    20
Diabetes     20
Anemia       20
Trombosis    20
Name: count, dtype: int64


In [11]:
# Crear un codificador de etiquetas para convertir las clases de texto en números
label_encoder = LabelEncoder()

# Convertir las clases de 'Enfermedad' a números
train_df['label'] = label_encoder.fit_transform(train_df['Enfermedad'])
val_df['label'] = label_encoder.transform(val_df['Enfermedad'])
test_df['label'] = label_encoder.transform(test_df['Enfermedad'])

train_df.head()

,Síntomas,Enfermedad,label
281,"He notado que mi pierna está hinchada, y las v...",Trombosis,3
22,"Estoy teniendo muchas infecciones, y mi piel s...",Anemia,0
146,"Últimamente me siento muy débil, y mi piel est...",Talasemia,2
70,He notado que el blanco de mis ojos tiene un t...,Anemia,0
376,"Tengo la vista borrosa, siempre tengo sed y pa...",Diabetes,1


## Entrenamiento

In [12]:
# Definimos la función de métricas
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "precision": precision_score(p.label_ids, preds, average='macro'),
        "recall": recall_score(p.label_ids, preds, average='macro'),
        "f1": f1_score(p.label_ids, preds, average='macro'),
    }

In [ ]:
# Definimos el modelo y el tokenizer
model_checkpoint = "distilbert-base-multilingual-cased"  
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

In [ ]:
# Preprocesamos los datos
def preprocess_function(examples):
    return tokenizer(examples['Text'], examples['Language'], padding=True, truncation=True, return_tensors="pt")

In [ ]:
# Converti i DataFrame in dataset di Hugging Face
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
validation_dataset = Dataset.from_pandas(val_df)
# Tokenizamos el dataset
train_preprocessed_dataset = train_dataset.map(preprocess_function, batched=True)
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)
val_preprocessed_dataset = validation_dataset.map(preprocess_function, batched=True)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=1e-5,              # Tasa de aprendizaje: se ha probado 5e-6, 1e-5, 5e-5
    per_device_train_batch_size=32,   # Tamaño del batch para entrenamiento: se ha probado 8, 16, 32, 64
    per_device_eval_batch_size=32,    # Tamaño del batch para evaluación
    num_train_epochs=3,
    weight_decay=0.3,               # Decaimiento de peso: hemos probado 0.01, 0.1, 0.2 e 0.3 e 0.4 y este era el mejor
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_preprocessed_dataset,
    eval_dataset=val_preprocessed_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics   # Función para calcular las métricas
)

## Training

In [ ]:
train_preprocessed_dataset

In [ ]:
# No modificar esta celda
# Esta celda, celda tiene que estar ejecutada en la entrega

start = time()

trainer.train()

end = time()
print(f">>>>>>>>>>>>> elapsed time: {(end-start)/60:.0f}m")

## Evaluación

In [ ]:
# Evaluación del modelo en el conjunto de prueba

# Preprocesamos el conjunto de prueba
test_preprocessed_dataset = test_dataset.map(preprocess_function, batched=True)

# Evaluamos el modelo en el conjunto de prueba
results = trainer.evaluate(test_preprocessed_dataset)

# Imprimimos las métricas de evaluación
print("Resultados de la evaluación en el conjunto de prueba:")
print(f"Exactitud: {results['eval_accuracy']:.4f}")
print(f"Precisión: {results['eval_precision']:.4f}")
print(f"Recall: {results['eval_recall']:.4f}")
print(f"F1-Score: {results['eval_f1']:.4f}")

# Para obtener más detalles sobre las predicciones y las métricas, también puedes obtener las predicciones:
# Realizamos predicciones en el conjunto de prueba
predictions = trainer.predict(test_preprocessed_dataset)

# Calculamos las métricas manualmente si lo deseas
preds = np.argmax(predictions.predictions, axis=1)
y_true = predictions.label_ids

# Exactitud
accuracy = accuracy_score(y_true, preds)
print(f"Exactitud calculada manualmente: {accuracy:.4f}")

# Precisión, recall y F1-score con el promedio macro
precision = precision_score(y_true, preds, average='macro')
recall = recall_score(y_true, preds, average='macro')
f1 = f1_score(y_true, preds, average='macro')

print(f"Precisión calculada manualmente: {precision:.4f}")
print(f"Recall calculado manualmente: {recall:.4f}")
print(f"F1-Score calculado manualmente: {f1:.4f}")

In [ ]:
# Salviamo il modello e il tokenizer
save_directory = os.path.join(MODELS_LOCATION, 'languiage_detection')
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)
print(f"Modello e tokenizer salvati nella directory {save_directory}")

In [ ]:
# Definiamo alcune frasi di esempio per il test che possano confondere il modello
sample_sentences = [
    {"Text": "Do you want tortilla for dinner?", "Language": "English"},
    {"Text": "Este es un ejemplo de software developement.", "Language": "Spanish"},
    {"Text": "Soy un cloud engineer, trabajo en google", "Language": "Spanish"}
]

# Convertiamo le frasi di esempio in un DataFrame
sample_df = pd.DataFrame(sample_sentences)

# Tokenizziamo le frasi di esempio
sample_dataset = Dataset.from_pandas(sample_df)
sample_preprocessed_dataset = sample_dataset.map(preprocess_function, batched=True)

# Effettuiamo le predizioni
sample_predictions = trainer.predict(sample_preprocessed_dataset)
sample_preds = np.argmax(sample_predictions.predictions, axis=1)
sample_df["Predicted Language"] = label_encoder.inverse_transform(sample_preds)

# Mostriamo i risultati
print("Risultati delle predizioni sulle frasi di esempio:")
sample_df